In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator
import rasterio
from rasterio.enums import Resampling
import cv2


nx, ny = 30,40
r_upsample = 25

test_image = np.arange(nx*ny).reshape(nx,ny)

In [14]:
def upsample_scipy(img,r):
    nx,ny = img.shape
    x = np.arange(nx)
    y = np.arange(ny)
    f = RegularGridInterpolator((x, y), img, method='cubic',bounds_error=False,fill_value=None)
    x_new = np.arange(0, nx, 1/r)
    y_new = np.arange(0, ny, 1/r)
    x_new, y_new = np.meshgrid(x_new, y_new, indexing='ij')
    return f((x_new, y_new))

In [16]:
%timeit upsample_scipy(test_image,r_upsample)

477 ms ± 3.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
def upsample_cv2(img, r):
    # img to the size cv2 expects
    img = np.uint16(img)
    # Determine the new size for the image
    new_size = (int(img.shape[1] * r), int(img.shape[0] * r))
    
    # Perform the resizing using cubic interpolation
    img_resized = cv2.resize(img, new_size, interpolation=cv2.INTER_CUBIC)
    
    return img_resized

In [34]:
%timeit upsample_cv2(test_image,r_upsample)

681 μs ± 9.29 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [35]:

def upsample_rasterio(tiff_path, r):
    
    with rasterio.open(tiff_path) as src:
        # resample data to target shape
        img_resampled = src.read(
            out_shape=(
                src.count,
                int(src.height * r),
                int(src.width * r)
            ),
            resampling=Resampling.cubic
        )
    
    return img_resampled


# create a new rasterio dataset from img
tif_path = 'test.tif'
with rasterio.open(tif_path, 'w', driver='GTiff', height=test_image.shape[0], width=test_image.shape[1], count=1, dtype=test_image.dtype) as dst:
    dst.write(test_image, 1)

/mnt/c/Users/tianr/Github/image_process_test/.venv/lib/python3.11/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


In [36]:
%timeit upsample_rasterio(tif_path,r_upsample)

/mnt/c/Users/tianr/Github/image_process_test/.venv/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


8.43 ms ± 170 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
